# 지금까지 배운 내용을 토대로 Open-Domain Question Answering(ODQA) 시스템 구축해보기


평가하고자 하는 Corpus: wikipedia 문서들

wikipedia 문서에서 뽑아낸 하나의 문서에 대해서 MRC 수행

## Requirements

In [ ]:
!pip install datasets==1.4.1 > /dev/null 2>&1 # execute command in silence
!pip install transformers==4.20.1 > /dev/null 2>&1
!pip install tqdm==4.41.1 > /dev/null 2>&1

In [ ]:
import random
import numpy as np
from pprint import pprint

from datasets import load_dataset, load_metric
from sklearn.feature_extraction.text import TfidfVectorizer

## 데이터 및 평가지표 불러오기

In [ ]:
# Load KLUE MRC dataset
from datasets import load_dataset
dataset = load_dataset("klue", 'mrc')

Couldn't find file locally at klue/klue.py, or remotely at https://raw.githubusercontent.com/huggingface/datasets/1.4.1/datasets/klue/klue.py.
The file was picked from the master branch on github instead at https://raw.githubusercontent.com/huggingface/datasets/master/datasets/klue/klue.py.


Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/mrc/1.0.0/55ff8f92b7a4b9842be6514ce0b4b5295b46d5e493f8bb5760da4be717018f90. Subsequent calls will reuse this data.


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers'],
        num_rows: 17554
    })
    validation: Dataset({
        features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers'],
        num_rows: 5841
    })
})

## Sparse retriever 가져오기

In [ ]:
# klue data가 포함된 wiki documents 가져오기
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1wVIgtc0YoQEwXB3JAsUud_86fRzrFCBd' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1wVIgtc0YoQEwXB3JAsUud_86fRzrFCBd" -O wikipedia_documents.json && rm -rf /tmp/cookies.txt

--2021-08-10 02:19:04--  https://docs.google.com/uc?export=download&confirm=ou16&id=1wVIgtc0YoQEwXB3JAsUud_86fRzrFCBd
Resolving docs.google.com (docs.google.com)... 108.177.97.138, 108.177.97.101, 108.177.97.102, ...
Connecting to docs.google.com (docs.google.com)|108.177.97.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-7c-docs.googleusercontent.com/docs/securesc/4sjc10qmj6rob1dlmarnnbn22gm0tfu6/uqj70qdjsbq8p305kjq4pabm9sqr2a0r/1628561925000/10000090644219567406/08378151277511945195Z/1wVIgtc0YoQEwXB3JAsUud_86fRzrFCBd?e=download [following]
--2021-08-10 02:19:05--  https://doc-04-7c-docs.googleusercontent.com/docs/securesc/4sjc10qmj6rob1dlmarnnbn22gm0tfu6/uqj70qdjsbq8p305kjq4pabm9sqr2a0r/1628561925000/10000090644219567406/08378151277511945195Z/1wVIgtc0YoQEwXB3JAsUud_86fRzrFCBd?e=download
Resolving doc-04-7c-docs.googleusercontent.com (doc-04-7c-docs.googleusercontent.com)... 74.125.204.132, 2404:6800:4008:c04::84
Connecting

In [ ]:
# First load wikipedia dump
import json

dump_path = 'wikipedia_documents.json' 
with open(dump_path, 'r') as f:
    wiki = json.load(f)

In [ ]:
# TODO: Extract & merge corpus from KLUE MRC dataset
from typing import List 
corpus: List[str] = ... # remove Type Hint first

In [ ]:
tokenizer_func = lambda x: x.split(' ')

vectorizer = TfidfVectorizer(tokenizer=tokenizer_func, ngram_range=(1,2))
# TODO: train and transform corpus to sparse matrix
sp_matrix = ...

In [ ]:
def get_relevant_doc(vectorizer, query, k=1):
    """
    참고: vocab 에 없는 이상한 단어로 query 하는 경우 assertion 발생 (예) 뙣뙇?
    """
    # TODO: transform query to query vector
    query_vec = ...
    assert np.sum(query_vec) != 0, "오류가 발생했습니다. 이 오류는 보통 query에 vectorizer의 vocab에 없는 단어만 존재하는 경우 발생합니다."
    result = query_vec * sp_matrix.T
    sorted_result = np.argsort(-result.data)
    doc_scores = result.data[sorted_result]
    doc_ids = result.indices[sorted_result]
    return doc_scores[:k], doc_ids[:k]

테스트해보기 
- 사용자 입력해서 어떤 문서가 나오는지 확인
- sample_idx 의 경우 4강에서 이미 진행

In [ ]:
""" 원하는 질문을 입력해보기 """
query = input("Enter any question: ") # "김영삼이 단일화를 시도한 평화민주당의 대선 후보 이름은?"

_, doc_id = get_relevant_doc(vectorizer, query, k=1)

""" 결과 확인 """

print("{} {} {}".format('*'*20, 'Result','*'*20))
print("[Search query]\n", query, "\n")
print(f"[Relevant Doc ID(Top 1 passage)]: {doc_id.item()}")
print(corpus[doc_id.item()])

# print(answer)

Enter any question: 김영삼이 단일화를 시도한 평화민주당의 대선 후보 이름은?
******************** Result ********************
[Search query]
 김영삼이 단일화를 시도한 평화민주당의 대선 후보 이름은? 

[Relevant Doc ID(Top 1 passage)]: 52892
결국 양김씨의 단일화 협상은 국민의 높은 열망에도 불구하고 결렬되었다.  김대중 의장은 민주당을 탈당하고 독자 대선 출마를 위한 신당 창당에 나섰으며, 김영삼 총재 역시 민주당의 대선 후보로 추대되기 위한 작업을 시작하였다.

민주당은 1987년 11월 9일 대의원 1,203명이 참석한 가운데 전당대회를 개최하고 김영삼 총재를 제13대 대통령 후보로 추대하였다.

김영삼 후보는 평화민주당의 대선 후보로 출마한 김대중 후보와 몇 차례에 걸쳐 단일화를 시도하였으나 실패하였고, 결국 야권은 분열된 채로 대선을 맞이하였다. 선거 결과 김영삼 후보와 김대중 후보는 각각 2위와 3위로 낙선하였으며, 민주정의당의 노태우 후보가 대통령에 당선되었다.


## 훈련된 MRC 모델 가져오기


In [ ]:
import torch
from transformers import (
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer
)

In [ ]:
model_name = 'sangrimlee/bert-base-multilingual-cased-korquad'
mrc_model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=True
)
mrc_model = mrc_model.eval()

In [ ]:
def get_answer_from_context(context, question, model, tokenizer):
    encoded_dict = tokenizer.encode_plus(  
        question,
        context,
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    non_padded_ids = encoded_dict["input_ids"][: encoded_dict["input_ids"].index(tokenizer.pad_token_id)]
    full_text = tokenizer.decode(non_padded_ids)
    inputs = {
    'input_ids': torch.tensor([encoded_dict['input_ids']], dtype=torch.long),
    'attention_mask': torch.tensor([encoded_dict['attention_mask']], dtype=torch.long),
    'token_type_ids': torch.tensor([encoded_dict['token_type_ids']], dtype=torch.long)
    }

    outputs = model(**inputs)

    # TODO: Using torch.max, get start and end position from start_logits & end_logits
    start = ...
    end = ...
    answer = tokenizer.decode(encoded_dict['input_ids'][start:end+1])
    return answer

In [ ]:
context = corpus[doc_id.item()]
answer = get_answer_from_context(context, query, mrc_model, tokenizer)
print("{} {} {}".format('*'*20, 'Result','*'*20))
print("[Search query]\n", query, "\n")
print(f"[Relevant Doc ID(Top 1 passage)]: {doc_id.item()}")
pprint(corpus[doc_id.item()], compact=True)
print(f"[Answer Prediction from the model]: {answer}")

******************** Result ********************
[Search query]
 김영삼이 단일화를 시도한 평화민주당의 대선 후보 이름은? 

[Relevant Doc ID(Top 1 passage)]: 52892
('결국 양김씨의 단일화 협상은 국민의 높은 열망에도 불구하고 결렬되었다.  김대중 의장은 민주당을 탈당하고 독자 대선 출마를 위한 신당 '
 '창당에 나섰으며, 김영삼 총재 역시 민주당의 대선 후보로 추대되기 위한 작업을 시작하였다.\n'
 '\n'
 '민주당은 1987년 11월 9일 대의원 1,203명이 참석한 가운데 전당대회를 개최하고 김영삼 총재를 제13대 대통령 후보로 '
 '추대하였다.\n'
 '\n'
 '김영삼 후보는 평화민주당의 대선 후보로 출마한 김대중 후보와 몇 차례에 걸쳐 단일화를 시도하였으나 실패하였고, 결국 야권은 분열된 채로 '
 '대선을 맞이하였다. 선거 결과 김영삼 후보와 김대중 후보는 각각 2위와 3위로 낙선하였으며, 민주정의당의 노태우 후보가 대통령에 '
 '당선되었다.')
[Answer Prediction from the model]: 김대중


## 통합해서 ODQA 시스템 구축하기

In [ ]:
def open_domain_qa(query, corpus, vectorizer, model, tokenizer, k=1):
    # 1. Retrieve k relevant docs by usign sparse matrix
    _, doc_id = get_relevant_doc(vectorizer, query, k=1)
    context = corpus[doc_id.item()]

    # 2. Predict answer from given doc by using MRC model
    answer = get_answer_from_context(context, query, mrc_model, tokenizer)
    print("{} {} {}".format('*'*20, 'Result','*'*20))
    print("[Search query]\n", query, "\n")
    print(f"[Relevant Doc ID(Top 1 passage)]: {doc_id.item()}")
    pprint(corpus[doc_id.item()], compact=True)
    print(f"[Answer Prediction from the model]: {answer}")

In [ ]:
query = input("Enter any question: ") #"김영삼이 단일화를 시도한 평화민주당의 대선 후보 이름은?"
open_domain_qa(query, corpus, vectorizer, mrc_model, tokenizer, k=1)

Enter any question: 김영삼이 단일화를 시도한 평화민주당의 대선 후보 이름은?
******************** Result ********************
[Search query]
 김영삼이 단일화를 시도한 평화민주당의 대선 후보 이름은? 

[Relevant Doc ID(Top 1 passage)]: 52892
('결국 양김씨의 단일화 협상은 국민의 높은 열망에도 불구하고 결렬되었다.  김대중 의장은 민주당을 탈당하고 독자 대선 출마를 위한 신당 '
 '창당에 나섰으며, 김영삼 총재 역시 민주당의 대선 후보로 추대되기 위한 작업을 시작하였다.\n'
 '\n'
 '민주당은 1987년 11월 9일 대의원 1,203명이 참석한 가운데 전당대회를 개최하고 김영삼 총재를 제13대 대통령 후보로 '
 '추대하였다.\n'
 '\n'
 '김영삼 후보는 평화민주당의 대선 후보로 출마한 김대중 후보와 몇 차례에 걸쳐 단일화를 시도하였으나 실패하였고, 결국 야권은 분열된 채로 '
 '대선을 맞이하였다. 선거 결과 김영삼 후보와 김대중 후보는 각각 2위와 3위로 낙선하였으며, 민주정의당의 노태우 후보가 대통령에 '
 '당선되었다.')
[Answer Prediction from the model]: 김대중
